In [11]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split, LeaveOneGroupOut, GridSearchCV, GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import shap

In [12]:
# Conjuntos de features
FEATURE_SETS = {
    'DS1': ['kilometer', 'wbgt', 'skin_temp', 'heart_rate', 'column_Male', 'age', 'vo2máx'],
    'DS2': ['kilometer', 'wbgt', 'skin_temp', 'heart_rate', 'age', 'column_Male', 'vo2máx', 'speed'],
    'DS3': ['kilometer', 'wbgt', 'skin_temp', 'heart_rate', 'age', 'column_Male', 'vo2máx', 'speed', 'umidade_absoluta', 'dry_temp', 'wet_temp', 'relative_humidity']
}


In [13]:
# Grades de hiperparâmetros para cada modelo
PARAM_GRIDS = {
    'Lasso': {
        'model__alpha': np.linspace(0.01, 0.1, 10),
    },
    'SVR': {
        'model__C': [0.1, 1.0, 10.0],
        'model__gamma': ['scale', 'auto'],
        'model__kernel': ['rbf', 'linear'],
    },
    'DecisionTreeRegressor': {
        'model__max_depth': [5, 10, 15, 20, None],
        'model__min_samples_leaf': [1, 5, 10, 20],
        'model__min_samples_split': [2, 5, 10],
    },
    # 'RandomForestRegressor': {
    #     'model__n_estimators': [100, 200, 400],
    #     'model__max_depth': [5, 10, 15],
    #     'model__min_samples_leaf': [5, 10, 20],
    #     'model__max_features': ['sqrt', 'log2', 0.5, 0.7],
    #     'model__max_samples': [0.7, 0.8, 0.9]
    # },
    # 'XGBRegressor': {
    #     'model__n_estimators': [300, 500, 700],
    #     'model__max_depth': [3, 5, 7],
    #     'model__learning_rate': [0.01, 0.05, 0.1],
    #     'model__subsample': [0.5, 0.7, 0.9],
    #     'model__colsample_bytree': [0.5, 0.7, 0.9],
    # }
}

In [14]:
# --- Funções para o Fluxo do Modelo ---

def load_and_split_data(X_path, y_path, groups_path, test_size=0.2, random_state=42):
    """
    Carrega os dados e os divide em conjuntos de treino e teste.
    Utiliza GroupShuffleSplit para garantir que os grupos não se misturem entre os conjuntos.
    """
    try:
        X = pd.read_csv(X_path)
        y = pd.read_csv(y_path)
        groups = pd.read_csv(groups_path)['trial_number']
    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado - {e.filename}. Verifique se os caminhos estão corretos.")
        return None, None, None, None, None, None

    splitter = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    train_idx, test_idx = next(splitter.split(X, y, groups=groups))

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx].values.ravel(), y.iloc[test_idx].values.ravel()
    groups_train = groups.iloc[train_idx]
    groups_test = groups.iloc[test_idx]

    return X_train, X_test, y_train, y_test, groups_train, groups_test


def run_grid_search(X_train, y_train, groups_train, feature_sets, param_grids):
    """
    Executa a busca em grade para múltiplos modelos, retornando os melhores resultados.
    """
    best_results = []
    scoring = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

    for model_name, param_grid in param_grids.items():
        print(f"\n--- Iniciando a Busca em Grade para o modelo: {model_name} ---")

        # Usando o conjunto de features DS2, que foi o melhor do notebook original
        feature_list = feature_sets['DS2']
        X_train_subset = X_train[feature_list]

        # Configura o pipeline com o modelo e o scaler se necessário
        if model_name in ['Lasso', 'SVR']:
            model = eval(f"{model_name}(" + "random_state=42)" if model_name != "SVR" else f"{model_name}()")
            pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('model', model)
            ])
        elif model_name == 'XGBRegressor':  
            model = xgb.XGBRegressor(random_state=42)
            pipeline = Pipeline([
                ('model', model)
            ])
            
        else:
            model = eval(f"{model_name}(" + "random_state=42)" if model_name != "SVR" else f"{model_name}()")
            pipeline = Pipeline([
                ('model', model)
            ])

        # Execução do GridSearchCV
        grid_search = GridSearchCV(
            pipeline,
            param_grid=param_grid,
            cv=LeaveOneGroupOut(),
            scoring=scoring,
            n_jobs=-1,
            verbose=1,
            return_train_score=True
        )
        grid_search.fit(X_train_subset, y_train, groups=groups_train)

        print(f"Busca em Grade para {model_name} Concluída.")
        print(f"Melhores hiperparâmetros: {grid_search.best_params_}")
        print(f"Melhor pontuação (RMSE) na validação cruzada: {-grid_search.best_score_:.4f}")

        best_results.append({
            'model': model_name,
            'best_params': grid_search.best_params_,
            'best_score': -grid_search.best_score_,
            'test_r2': r2_score(y_test, grid_search.best_estimator_.predict(X_test[feature_list])),
            'test_rmse': np.sqrt(mean_squared_error(y_test, grid_search.best_estimator_.predict(X_test[feature_list])))
        })
    
    return best_results


def evaluate_final_model(best_results, X_test, y_test, feature_sets):
    """
    Avalia o melhor modelo de todos os grid searches no conjunto de teste (hold-out).
    """
    # Encontra o modelo com o melhor score geral
    best_overall_model_name = min(best_results, key=lambda x: x['best_score'])['model']
    best_params = min(best_results, key=lambda x: x['best_score'])['best_params']
    best_feature_set = feature_sets['DS2'] # O DS2 foi o melhor no notebook original
    X_test_best = X_test[best_feature_set]

    # Re-instancia o melhor modelo com os melhores parâmetros
    if best_overall_model_name in ['Lasso', 'SVR']:
        final_model_instance = eval(f"{best_overall_model_name}()")
        final_pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', final_model_instance)
        ])
    elif best_overall_model_name == 'XGBRegressor':
        final_model_instance = xgb.XGBRegressor(random_state=42)
        final_pipeline = Pipeline([
            ('model', final_model_instance)
        ])  

    else:
        final_model_instance = eval(f"{best_overall_model_name}()")
        final_pipeline = Pipeline([
            ('model', final_model_instance)
        ])
    
    # Define os melhores parâmetros no pipeline
    final_pipeline.set_params(**best_params)
    
    print(f"\n--- Avaliando o melhor modelo no conjunto de teste: {best_overall_model_name} ---")
    
    # AVALIANDO O MODELO AQUI COM DADOS QUE ELE NUNCA VIU
    final_pipeline.fit(X_test_best, y_test)  # Treina o modelo com os dados de teste (hold-out)
    y_test_pred = final_pipeline.predict(X_test_best)
    r2_final = r2_score(y_test, y_test_pred)
    rmse_final = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    print(f" - R²: {r2_final:.4f}")
    print(f" - RMSE: {rmse_final:.4f}")


In [18]:
# --- Função Principal de Execução ---

if __name__ == '__main__':
    # 1. Carregar e dividir os dados
    print("--- Carregando e dividindo os dados ---")
    X_train, X_test, y_train, y_test, groups_train, groups_test = load_and_split_data(
        X_path='/home/usuario-leticia/Desktop/Samuel/leticiaag/tcore/data/processed-data/X-data1-1km.csv',
        y_path='/home/usuario-leticia/Desktop/Samuel/leticiaag/tcore/data/processed-data/y-data1-1km.csv',
        groups_path='/home/usuario-leticia/Desktop/Samuel/leticiaag/tcore/data/processed-data/groups-data1-1km.csv'
    )
    if X_train is None:
        exit()

    # 2. Executar a busca em grade para todos os modelos
    best_results_df = run_grid_search(X_train, y_train, groups_train, FEATURE_SETS, PARAM_GRIDS)

    # 3. Avaliar o melhor modelo geral no conjunto de teste
    evaluate_final_model(best_results_df, X_test, y_test, FEATURE_SETS)
    
    # 4. Exibir os resultados finais
    results_df = pd.DataFrame(best_results_df)



    print(results_df)

    


--- Carregando e dividindo os dados ---

--- Iniciando a Busca em Grade para o modelo: Lasso ---
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Busca em Grade para Lasso Concluída.
Melhores hiperparâmetros: {'model__alpha': np.float64(0.030000000000000006)}
Melhor pontuação (RMSE) na validação cruzada: 0.3913

--- Iniciando a Busca em Grade para o modelo: SVR ---
Fitting 60 folds for each of 12 candidates, totalling 720 fits
Busca em Grade para SVR Concluída.
Melhores hiperparâmetros: {'model__C': 0.1, 'model__gamma': 'scale', 'model__kernel': 'linear'}
Melhor pontuação (RMSE) na validação cruzada: 0.4040

--- Iniciando a Busca em Grade para o modelo: DecisionTreeRegressor ---
Fitting 60 folds for each of 60 candidates, totalling 3600 fits
Busca em Grade para DecisionTreeRegressor Concluída.
Melhores hiperparâmetros: {'model__max_depth': 5, 'model__min_samples_leaf': 20, 'model__min_samples_split': 2}
Melhor pontuação (RMSE) na validação cruzada: 0.4526

--- Avaliando o

In [ ]:
###TESTE


# a diferença entre os modelos superior e inferior é de que no inferior, os modelos sao otimizados para cada data set poposto



In [39]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split, LeaveOneGroupOut, GridSearchCV, GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import shap

# --- Definição de Constantes e Hiperparâmetros ---
# Conjuntos de features
FEATURE_SETS = {
    'DS1': ['kilometer', 'wbgt', 'skin_temp', 'heart_rate', 'column_Male', 'age', 'vo2máx'],
    'DS2': ['kilometer', 'wbgt', 'skin_temp', 'heart_rate', 'age', 'column_Male', 'vo2máx', 'speed'],
    'DS3': ['kilometer', 'wbgt', 'skin_temp', 'heart_rate', 'age', 'column_Male', 'vo2máx', 'speed', 'umidade_absoluta', 'dry_temp', 'wet_temp', 'relative_humidity']
}

# Grades de hiperparâmetros para cada modelo
PARAM_GRIDS = {
    'Lasso': {
        'model__alpha': np.linspace(0.01, 0.1, 10),
    },
    'SVR': {
        'model__C': [0.1, 1.0],
        'model__gamma': ['scale', 'auto'],
        'model__kernel': ['rbf', 'linear'],
    },
    'DecisionTree': {
        'model__max_depth': [5, 10],
        'model__min_samples_leaf': [1, 5],
        'model__min_samples_split': [2, 5],
    },
    'RandomForest': {
        'model__n_estimators': [100, 200],
        'model__max_depth': [5, 10],
        # 'model__min_samples_leaf': [5, 10, 20],
        # 'model__max_features': ['sqrt', 'log2', 0.5, 0.7],
        # 'model__max_samples': [0.7, 0.8, 0.9]
    },
    'XGBoost': {
        'model__n_estimators': [300, 500, 700],
        'model__max_depth': [3, 5, 7],
        # 'model__learning_rate': [0.01, 0.05, 0.1],
        # 'model__subsample': [0.5, 0.7, 0.9],
        # 'model__colsample_bytree': [0.5, 0.7, 0.9],
    }
}

# Dicionário para mapear os nomes dos modelos para as suas classes
MODELS_TO_RUN = {
    'Lasso': Lasso(random_state=42, max_iter=20000),
    'SVR': SVR(),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': xgb.XGBRegressor(random_state=42, objective='reg:squarederror')
}

# --- Funções para o Fluxo do Modelo ---

def load_and_split_data(X_path, y_path, groups_path, test_size=0.2, random_state=42):
    """
    Carrega os dados e os divide em conjuntos de treino e teste.
    Utiliza GroupShuffleSplit para garantir que os grupos não se misturem entre os conjuntos.
    """
    try:
        X = pd.read_csv(X_path)
        y = pd.read_csv(y_path)
        groups = pd.read_csv(groups_path)['trial_number']
    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado - {e.filename}. Verifique se os caminhos estão corretos.")
        return None, None, None, None, None, None

    splitter = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    train_idx, test_idx = next(splitter.split(X, y, groups=groups))

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx].values.ravel(), y.iloc[test_idx].values.ravel()
    groups_train = groups.iloc[train_idx]
    groups_test = groups.iloc[test_idx]

    return X_train, X_test, y_train, y_test, groups_train, groups_test


def run_grid_search_and_evaluate(X_train, y_train, groups_train, X_test, y_test, feature_sets, param_grids, models_to_run):
    """
    Executa a busca em grade para múltiplos modelos e conjuntos de features,
    e avalia o melhor modelo de cada busca.
    """
    best_results = []
    scoring = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

    for model_name, model_instance in models_to_run.items():
        print(f"\n--- Iniciando a Busca em Grade para o modelo: {model_name} ---")

        for feature_set_name, feature_list in feature_sets.items():
            print(f"\n--- Otimizando com o conjunto de features: {feature_set_name} ---")
            
            # Subconjunto dos dados com as features atuais
            X_train_subset = X_train[feature_list]
            X_test_subset = X_test[feature_list]

            # Configura o pipeline com o scaler se necessário
            if model_name in ['Lasso', 'SVR']:
                pipeline = Pipeline([
                    ('scaler', StandardScaler()),
                    ('model', model_instance)
                ])
            else:
                pipeline = Pipeline([
                    ('model', model_instance)
                ])
            
            param_grid = param_grids.get(model_name)
            if not param_grid:
                print(f"Aviso: Grade de hiperparâmetros não encontrada para {model_name}. Pulando este modelo.")
                continue

            # Execução do GridSearchCV
            grid_search = GridSearchCV(
                pipeline,
                param_grid=param_grid,
                cv=LeaveOneGroupOut(),
                scoring=scoring,
                n_jobs=-1,
                verbose=1,
                return_train_score=True
            )
            grid_search.fit(X_train_subset, y_train, groups=groups_train)

            print(f"Busca em Grade para {model_name} com {feature_set_name} Concluída.")
            print(f"Melhores hiperparâmetros: {grid_search.best_params_}")
            print(f"Melhor pontuação (RMSE) na validação cruzada: {-grid_search.best_score_:.4f}")

            # Avaliação no conjunto de teste
            y_test_pred = grid_search.best_estimator_.predict(X_test_subset)
            r2_final = r2_score(y_test, y_test_pred)
            rmse_final = np.sqrt(mean_squared_error(y_test, y_test_pred))

            print(f" - R² no conjunto de teste: {r2_final:.4f}")
            print(f" - RMSE no conjunto de teste: {rmse_final:.4f}")

             # Avaliação no conjunto de treino
            y_train_pred = grid_search.best_estimator_.predict(X_train_subset)
            r2_train = r2_score(y_train, y_train_pred)
            rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))

            # Avaliação no conjunto de teste
            y_test_pred = grid_search.best_estimator_.predict(X_test_subset)
            r2_final = r2_score(y_test, y_test_pred)
            rmse_final = np.sqrt(mean_squared_error(y_test, y_test_pred))

            best_results.append({
                'model': model_name,
                'feature_set': feature_set_name, # Novo campo para identificar o conjunto de features
                'best_params': grid_search.best_params_,
                'best_cv_score': -grid_search.best_score_,
                'test_r2': r2_final,
                'train_r2': r2_train,           # <-- NOVO
                'test_rmse': rmse_final,
                'train_rmse': rmse_train       # <-- NOVO
            })
    
    return pd.DataFrame(best_results)


# --- Função Principal de Execução ---

if __name__ == '__main__':
    # 1. Carregar e dividir os dados
    print("--- Carregando e dividindo os dados ---")
    X_train, X_test, y_train, y_test, groups_train, groups_test = load_and_split_data(
        X_path='/home/usuario-leticia/Desktop/Samuel/leticiaag/tcore/data/processed-data/X-data1-1km.csv',
        y_path='/home/usuario-leticia/Desktop/Samuel/leticiaag/tcore/data/processed-data/y-data1-1km.csv',
        groups_path='/home/usuario-leticia/Desktop/Samuel/leticiaag/tcore/data/processed-data/groups-data1-1km.csv'
    )
    # Garante que os dados foram carregados antes de prosseguir
    if X_train is None:
        exit()
        
    print("Dados carregados e divididos com sucesso.")

    # 2. Executar a busca em grade para todos os modelos
    print("\n--- Iniciando a Busca em Grade para todos os algoritmos ---")
    results_df = run_grid_search_and_evaluate(
        X_train,
        y_train,
        groups_train,
        X_test,
        y_test,
        FEATURE_SETS,
        PARAM_GRIDS,
        MODELS_TO_RUN
    )
    print("\n--- Resultados da Busca em Grade Concluída ---") 
    # 3. Exibir os resultados finais
    display(results_df.round(4))


--- Carregando e dividindo os dados ---
Dados carregados e divididos com sucesso.

--- Iniciando a Busca em Grade para todos os algoritmos ---

--- Iniciando a Busca em Grade para o modelo: Lasso ---

--- Otimizando com o conjunto de features: DS1 ---
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Busca em Grade para Lasso com DS1 Concluída.
Melhores hiperparâmetros: {'model__alpha': np.float64(0.030000000000000006)}
Melhor pontuação (RMSE) na validação cruzada: 0.3899
 - R² no conjunto de teste: 0.8404
 - RMSE no conjunto de teste: 0.4068

--- Otimizando com o conjunto de features: DS2 ---
Fitting 60 folds for each of 10 candidates, totalling 600 fits
Busca em Grade para Lasso com DS2 Concluída.
Melhores hiperparâmetros: {'model__alpha': np.float64(0.030000000000000006)}
Melhor pontuação (RMSE) na validação cruzada: 0.3913
 - R² no conjunto de teste: 0.8483
 - RMSE no conjunto de teste: 0.3966

--- Otimizando com o conjunto de features: DS3 ---
Fitting 60 folds for eac

,model,feature_set,best_params,best_cv_score,test_r2,train_r2,test_rmse,train_rmse
0,Lasso,DS1,{'model__alpha': 0.030000000000000006},0.3899,0.8404,0.8098,0.4068,0.4096
1,Lasso,DS2,{'model__alpha': 0.030000000000000006},0.3913,0.8483,0.8119,0.3966,0.4074
2,Lasso,DS3,{'model__alpha': 0.030000000000000006},0.3919,0.8492,0.8123,0.3955,0.4069
3,SVR,DS1,"{'model__C': 0.1, 'model__gamma': 'scale', 'mo...",0.4024,0.8343,0.8107,0.4145,0.4086
4,SVR,DS2,"{'model__C': 0.1, 'model__gamma': 'scale', 'mo...",0.4040,0.8540,0.8154,0.3891,0.4035
5,SVR,DS3,"{'model__C': 0.1, 'model__gamma': 'scale', 'mo...",0.4132,0.8545,0.8151,0.3885,0.4038
6,DecisionTree,DS1,"{'model__max_depth': 10, 'model__min_samples_l...",0.4564,0.8126,0.9181,0.4409,0.2687
7,DecisionTree,DS2,"{'model__max_depth': 5, 'model__min_samples_le...",0.4611,0.8149,0.8562,0.4382,0.3561
8,DecisionTree,DS3,"{'model__max_depth': 5, 'model__min_samples_le...",0.4611,0.8045,0.8562,0.4503,0.3561
9,RandomForest,DS1,"{'model__max_depth': 10, 'model__n_estimators'...",0.3674,0.8735,0.9763,0.3622,0.1447
